In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
np.set_printoptions(precision=2)

from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
path = '/content/drive/My Drive/dax_ml/orgdat_ab240409_size/cnv_org2trd/clean_std3/'

---
---
---
## Convert Trade-File
(time, vol, price) to Score-File (time, score, +prepare various Attributes for Learning).  
path: /content/drive/My Drive/dax_ml/orgdat_ab240409_size/cnv_org2trd/clean_std/     
In:  path + name + '.csv'  
Out: path + 'cnv_trd2score/'+name+'Score.csv'  


In [ ]:
def convert_trade2score(path, name):
  fileOut = open(path + 'cnv_trd2score/'+name+'Score.csv', 'w')
  ar = np.genfromtxt(path + name + '.txt.csv' , delimiter=',', skip_header=1, dtype=int)  #.txt.csv

  futur0 =   50
  futur1 =  250
  futur3 =  350    #define for trigger in the middle
  futur2 = 1000
  #firstPrice = ar[0, 1]    #keep first price
  scoreSize  = 25          #number of prices per score  (first try 5)
  avgSize    = 50

  rowanz, colanz = ar.shape
  endloop = rowanz-1100
  acti  = futur2 + scoreSize + 10  #0  start in future > back view
  lastTrigger250,  lastTrigger500, lastTrigger1000 = 0, 0, 0
  fileOut.write("timeLastStr,timediff,timescore,prcdiff,prcgrad,volSum,timeLast,priceLast,priceAvgXX,priceMean,priceMin,priceMax,priceScore,")
  fileOut.write("timein50Str,timein50,pricein50,min50,max50,")
  fileOut.write("timein250Str,timein250,pricein250,min250,max250,")
  fileOut.write("timein500Str,timein500,pricein500,min500,max500,")
  fileOut.write("timein1000Str,timein1000,pricein1000,min1000,max1000,")
  fileOut.write("ago250timeStr,ago250time,ago250price,ago250diff,ago250grad,ago250diffAct,avg250min,avg250max,avg250siz,avg250wavg,avg250diff,avg250grad,avg250diffAct,avg250crossAct,")
  fileOut.write("ago1000timeStr,ago1000time,ago1000price,ago1000diff,ago1000grad,ago1000diffAct,avg1000min,avg1000max,avg1000siz,avg1000wavg,avg1000diff,avg1000grad,avg1000diffAct,avg1000crossAct,")
  fileOut.write("trigger250,trigger500,trigger1000\n")
  while acti < endloop:    #until end of arr

    lasti_np = acti+scoreSize                       #lasti np arr is excl
    lasti    = lasti_np - 1
    prclast  =  ar[lasti , 1]
    prcdiff  =  prclast - ar[acti , 1]
    timediff = (ar[lasti , 0] - ar[acti , 0])/1000
    prcgrad  = prcdiff/timediff
    timescore= 3/timediff
    volsum   = ar[ acti:lasti_np , 2].sum()
    volscore = volsum / (scoreSize*2)

    fileOut.write(""  + str(pd.to_datetime(ar[ lasti , 0], unit='ms')))
    fileOut.write("," + str(timediff))
    fileOut.write("," + str(timescore))          #timescore
    fileOut.write("," + str(prcdiff))
    fileOut.write("," + str(prcgrad))            #prcgrad : prc per time (=steigung/gradent)
    fileOut.write("," + str(volsum))

    fileOut.write("," + str(ar[ lasti , 0]))                       #timelast
    fileOut.write("," + str(prclast))                              #price last
    priceAvgXX = ar[ acti-avgSize:lasti_np , 1].mean()
    fileOut.write("," + str(int(priceAvgXX)))                      #priceAvgXX
    fileOut.write("," + str(int(ar[ acti:lasti_np , 1].mean())))   #priceMean
    priceMin = ar[ acti:lasti_np , 1].min()
    priceMax = ar[ acti:lasti_np , 1].max()
    fileOut.write("," + str(int( priceMin )))    # priceMin
    fileOut.write("," + str(int( priceMax )))    # priceMax
    fileOut.write("," + str(int(prcdiff*volscore*timescore)))   #priceScore

    fileOut.write("," + str(pd.to_datetime(ar[ lasti+futur0 , 0], unit='ms')))   # in 50
    fileOut.write("," + str(ar[ lasti+futur0 , 0]))                         #time  in 50
    fileOut.write("," + str(ar[ lasti+futur0 , 1]))                         #price in 50
    fileOut.write("," + str(int(ar[ lasti:lasti_np+futur0 , 1].min())))
    fileOut.write("," + str(int(ar[ lasti:lasti_np+futur0 , 1].max())))

    fileOut.write("," + str(pd.to_datetime(ar[ lasti+futur1 , 0], unit='ms')))   # in 250
    fileOut.write("," + str(ar[ lasti+futur1 , 0]))                          #time in 250
    fileOut.write("," + str(ar[ lasti+futur1 , 1]))                         #price in 250
    min250  = ar[ lasti:lasti_np+futur1 , 1].min()
    max250  = ar[ lasti:lasti_np+futur1 , 1].max()
    fileOut.write("," + str(int( min250 )))                      # min250
    fileOut.write("," + str(int( max250 )))                      # max250

    fileOut.write("," + str(pd.to_datetime(ar[ lasti+futur3 , 0], unit='ms')))   # in 500
    fileOut.write("," + str(ar[ lasti+futur3 , 0]))                          #time in 500
    fileOut.write("," + str(ar[ lasti+futur3 , 1]))                         #price in 500
    min500  = ar[ lasti:lasti_np+futur3 , 1].min()
    max500  = ar[ lasti:lasti_np+futur3 , 1].max()
    fileOut.write("," + str(int( min500 )))                      # min500
    fileOut.write("," + str(int( max500 )))                      # max500

    fileOut.write("," + str(pd.to_datetime(ar[ lasti+futur2 , 0], unit='ms')))     #in 1000
    fileOut.write("," + str(ar[ lasti+futur2 , 0]))                           #time in 1000
    fileOut.write("," + str(ar[ lasti+futur2 , 1]))                          #price in 1000
    min1000 = ar[ lasti:lasti_np+futur2 , 1].min()
    max1000 = ar[ lasti:lasti_np+futur2 , 1].max()
    fileOut.write("," + str(int( min1000 )))                      # min1000
    fileOut.write("," + str(int( max1000 )))                      # max1000

    # ########################################################################
    # --- davor werte 250 (also inhalte wie 1 block davor)
    timago250      = ar[ lasti-futur1 , 0]
    timago250davor = ar[ lasti-futur1-scoreSize , 0]
    fileOut.write("," + str(pd.to_datetime(timago250, unit='ms')))     # ######### ago 250
    fileOut.write("," + str(timago250))                #time ago 250
    prcago250      = ar[ acti-futur1          :lasti_np-futur1           , 1].mean()    # ar[ lasti-futur1, 1]
    prcago250davor = ar[ acti-futur1-scoreSize:lasti_np-futur1-scoreSize , 1].mean()    #ar[ lasti-futur1-scoreSize, 1]
    fileOut.write("," + str( int( prcago250 )))        #price ago 250
    ago250diff     =  prcago250 - prcago250davor
    fileOut.write("," + str(int( ago250diff )))        #ago250diff
    ago250grad     = ago250diff / (timago250-timago250davor)
    fileOut.write("," + str(ago250grad ))              #ago250grad
    fileOut.write("," + str(int( prcago250 - priceAvgXX )))  #ago250diffAct

    avg250min = ar[ lasti_np-futur1:lasti , 1].min()
    avg250max = ar[ lasti_np-futur1:lasti , 1].max()
    fileOut.write("," + str(int( avg250min )))         #avg250min  (min im avg zeitraum)
    fileOut.write("," + str(int( avg250max )))         #avg250max  (max ...)
    avg250siz      = ar[ lasti_np-futur1           :lasti , 2].sum()
    avg250sizdavor = ar[ lasti_np-futur1-scoreSize :lasti-scoreSize , 2].sum()
    fileOut.write("," + str(int(avg250siz)))           #avg250siz

    avg250wavg        = (ar[ lasti_np-futur1           :lasti           , 1] * ar[ lasti_np-futur1          :lasti           , 2]).sum() / avg250siz
    avg250wavgdavor   = (ar[ lasti_np-futur1-scoreSize :lasti-scoreSize , 1] * ar[ lasti_np-futur1-scoreSize:lasti-scoreSize , 2]).sum() / avg250sizdavor
    fileOut.write("," + str(int( avg250wavg )))        #avg250wavg

    # --- avg werte 250 (werte auf avg zeitraum)
    avg250diff     =  avg250wavg - avg250wavgdavor
    fileOut.write("," + str(int( avg250diff )))         #avg250diff
    avg250grad     = avg250diff / timediff
    fileOut.write("," + str( avg250grad ))              #avg250grad
    fileOut.write("," + str(int( avg250wavg - priceAvgXX )))  #avg250diffAct  (not prclast)
    # --- cross 250
    ago250crossAct =  0    # (-1 wenn von unten nach oben, 1 wenn von oben nach unten)
    if    avg250wavgdavor <= priceAvgXX and avg250wavg > priceAvgXX : avg250crossAct = -1  # (npt prclast)
    elif  avg250wavgdavor >= priceAvgXX and avg250wavg < priceAvgXX : avg250crossAct =  1
    fileOut.write("," + str(int( ago250crossAct )))

    # ########################################################################
    # --- davor werte 1000 (also inhalte wie 1 block davor)
    timago1000      = ar[ lasti-futur2 , 0]
    timago1000davor = ar[ lasti-futur2-scoreSize , 0]
    fileOut.write("," + str(pd.to_datetime(timago1000, unit='ms')))     # ######### ago 1000
    fileOut.write("," + str(timago1000))               #time ago 1000
    prcago1000      = ar[ acti-futur2          :lasti_np-futur2           , 1].mean()    #ar[ lasti-futur2, 1]
    prcago1000davor = ar[ acti-futur2-scoreSize:lasti_np-futur2-scoreSize , 1].mean()    #ar[ lasti-futur2-scoreSize, 1]
    fileOut.write("," + str( int( prcago1000 )))       #price ago 1000
    ago1000diff     =  prcago1000 - prcago1000davor
    fileOut.write("," + str(int( ago1000diff )))       #ago1000diff
    ago1000grad     = ago250diff / (timago250-timago250davor)
    fileOut.write("," + str(ago1000grad ))             #ago1000grad
    fileOut.write("," + str(int( prcago1000 - priceAvgXX )))  #ago1000diffAct

    avg1000min = ar[ lasti_np-futur2:lasti , 1].min()
    avg1000max = ar[ lasti_np-futur2:lasti , 1].max()
    fileOut.write("," + str(int( avg1000min )))        #avg1000min  (min im avg zeitraum)
    fileOut.write("," + str(int( avg1000max )))        #avg1000max  (max ...)
    avg1000siz      = ar[ lasti_np-futur2           :lasti , 2].sum()
    avg1000sizdavor = ar[ lasti_np-futur2-scoreSize :lasti-scoreSize , 2].sum()
    fileOut.write("," + str(int(avg1000siz)))          #avg1000siz

    avg1000wavg        = (ar[ lasti_np-futur2           :lasti           , 1] * ar[ lasti_np-futur2          :lasti           , 2]).sum() / avg1000siz
    avg1000wavgdavor   = (ar[ lasti_np-futur2-scoreSize :lasti-scoreSize , 1] * ar[ lasti_np-futur2-scoreSize:lasti-scoreSize , 2]).sum() / avg1000sizdavor
    fileOut.write("," + str(int( avg1000wavg )))       #avg1000wavg

    # --- avg werte 1000 (werte auf avg zeitraum)
    avg1000diff    =  avg1000wavg - avg1000wavgdavor
    fileOut.write("," + str(int( avg1000diff )))       #avg1000diff
    avg1000grad    = avg1000diff / timediff
    fileOut.write("," + str( avg1000grad ))            #avg1000grad
    fileOut.write("," + str(int( avg1000wavg - priceAvgXX )))  #avg1000diffAct
    # --- cross 250
    ago1000crossAct =  0    # (-1 wenn von unten nach oben, 1 wenn von oben nach unten)
    if    avg1000wavgdavor <= priceAvgXX and avg1000wavg > priceAvgXX : avg1000crossAct = -1
    elif  avg1000wavgdavor >= priceAvgXX and avg1000wavg < priceAvgXX : avg1000crossAct =  1
    fileOut.write("," + str(int( ago1000crossAct )))

    # ########################################################################
    # --- trigger250  #  (wenn 250 min/max ist erreicht, dann trigger +1/-1 )
    if    priceMin <= min250 and lastTrigger250 !=  1: trigger250 =  1; lastTrigger250 =  1
    elif  priceMax >= max250 and lastTrigger250 != -1: trigger250 = -1; lastTrigger250 = -1
    else: trigger250 = 0
    fileOut.write("," + str(int( trigger250 )))
    # --- trigger500  #  (wenn 500 min/max ist erreicht, dann trigger +1/-1 )
    if    priceMin <= min500 and lastTrigger500 !=  1: trigger500 =  1; lastTrigger500 =  1
    elif  priceMax >= max500 and lastTrigger500 != -1: trigger500 = -1; lastTrigger500 = -1
    else: trigger500 = 0
    fileOut.write("," + str(int( trigger500 )))
    # --- trigger1000  # (wenn 1000 min/max ist erreicht, dann trigger +1/-1 )
    if    priceMin <= min1000 and lastTrigger1000 !=  1 : trigger1000 =  1; lastTrigger1000 =  1
    elif  priceMax >= max1000 and lastTrigger1000 != -1 : trigger1000 = -1; lastTrigger1000 = -1
    else: trigger1000 =  0
    fileOut.write("," + str(int( trigger1000 )))

    fileOut.write("\n")
    acti = acti + scoreSize

  fileOut.close()
  #print("write ende score: "  + name)

In [ ]:
def extend_trade2score(path_, name):
  df3c = pd.read_csv(path_ +  'cnv_trd2score/'+name+'Score.csv')
  daybas  = df3c["priceLast"].head(1).values
  #print(daybas)
  df3c["priceMin0"]   =df3c["priceMin"]   -daybas
  df3c["priceMax0"]   =df3c["priceMax"]   -daybas
  df3c["priceMean0"]  =df3c["priceAvgXX"]   -daybas
  df3c["avg250wavg0"] =df3c["avg250wavg"]   -daybas
  df3c["avg1000wavg0"]=df3c["avg1000wavg"]  -daybas
  df3c["pricein250n"] =df3c["pricein250"]   -daybas
  df3c["pricein1000n"]=df3c["pricein1000"]  -daybas
  df3c["avg250wavgDif"] =df3c["avg250wavg"]   - df3c["priceAvgXX"]
  df3c["avg1000wavgDif"]=df3c["avg1000wavg"]  - df3c["priceAvgXX"]
  df3c["pricein250dif"] =df3c["pricein250"]   - df3c["priceAvgXX"]
  df3c["min250_"]     =df3c["min250"]     -daybas
  df3c["max250_"]     =df3c["max250"]     -daybas
  df3c["min500_"]     =df3c["min500"]     -daybas
  df3c["max500_"]     =df3c["max500"]     -daybas
  df3c["min1000_"]    =df3c["min1000"]    -daybas
  df3c["max1000_"]    =df3c["max1000"]    -daybas
  #f3c["timems"] = pd.to_datetime(df3c["time"], unit='ms')

  df3c.to_csv(path + 'cnv_trd2score/extnd/'+name+'Ext.csv' )
  #print("write ende extnd: "  + name)

In [ ]:
for name in os.listdir(path):
  if name.endswith(".txt.csv"): #.txt.csv
    convert_trade2score(path, name[:-8])   # -8
    extend_trade2score (path, name[:-8])
    print("write ende: "  + name)